# Graveyard Of Wikidata - Statistics

Github : https://github.com/haikelJebahi/Graveyard-of-wikidata

Data : https://iccl.inf.tu-dresden.de/web/Wikidata_SPARQL_Logs/en

## Imports

In [1]:
%matplotlib inline
from joblib import dump, load
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from urllib.parse import unquote_plus
from pyparsing import ParseException
from rdflib.plugins.sparql.algebra import translateQuery, translateUpdate, pprintAlgebra, traverse
from rdflib.plugins.sparql.parser import parseQuery, parseUpdate
from rdflib.plugins.sparql.operators import Builtin_LANG
from rdflib.plugins.sparql.parserutils import CompValue
from rdflib.term import Variable
from pandas import concat
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import colorcet as cc
from sklearn.cluster import AgglomerativeClustering
from PIL import Image
import io
import gc

In [2]:
plt.style.use("ggplot")

In [3]:
pd.set_option("display.max_rows", 0)
pd.set_option("display.max_columns", 0)
pd.set_option("display.expand_frame_repr", True)
pd.set_option("display.max_colwidth", -1)

/var/folders/lm/zh_hww991gd26t0d10kfp2km0000gn/T/ipykernel_388/3536214460.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


## Statistics

In [4]:
nom = "../data/merged/status500_Joined.parsed.tsv"
df_nona = pd.read_csv(nom, sep="\t")


FileNotFoundError: [Errno 2] No such file or directory: '../data/merged/status500_Joined.parsed.tsv'

Adding basic attributes of queries

In [ ]:
df_nona["has_variable"] = df_nona["var_cpt"].apply(lambda x: False if x == 0 else True)
df_nona["has_filter"] = df_nona["filter"].apply(lambda x: False if x == 0 else True)
df_nona["has_orderby"] = df_nona["orderby"].apply(lambda x: False if x == 0 else True)
df_nona["has_limit"] = df_nona["limit"].apply(lambda x: False if x == 0 else True)
df_nona["has_select"] = df_nona["select"].apply(lambda x: False if x == 0 else True)
df_nona["has_distinct"] = df_nona["distinct"].apply(lambda x: False if x == 0 else True)
df_nona["has_modify"] = df_nona["modify"].apply(lambda x: False if x == 0 else True)
df_nona["has_tomultiset"] = df_nona["tomultiset"].apply(
    lambda x: False if x == 0 else True
)
df_nona["has_join"] = df_nona["join"].apply(lambda x: False if x == 0 else True)
df_nona["has_bgp"] = df_nona["bgp"].apply(lambda x: False if x == 0 else True)
df_nona["has_project"] = df_nona["project"].apply(lambda x: False if x == 0 else True)
df_nona["has_union"] = df_nona["union"].apply(lambda x: False if x == 0 else True)
df_nona["has_values"] = df_nona["values"].apply(lambda x: False if x == 0 else True)
df_nona["has_slice"] = df_nona["slice"].apply(lambda x: False if x == 0 else True)
df_nona["has_offset"] = df_nona["offset"].apply(lambda x: False if x == 0 else True)

df_nona["complexPathWith"] = df_nona.loc[:,['pathWithStar','pathWithQuestionMark','pathWithPlus']].sum(axis=1)
df_nona["simplePathWith"] = df_nona.loc[:,['pathWithInv','pathWithSequence','pathWithAlternative']].sum(axis=1)

In [ ]:
df_nona

Split in robotic and organic

In [ ]:
df_nona_robotic = df_nona.loc[(df_nona["sourceCategory"] == "robotic")]
df_nona_robotic


In [ ]:
df_nona_organic = df_nona.loc[(df_nona["sourceCategory"] == "organic")]
df_nona_organic


In [ ]:
len(df_nona)


In [ ]:

data = dict()
for i in range(1,8):
    df_nona_interval = df_nona[df_nona["interval"] == i]
    for type in ["robotic", "organic"]:
        df_nona_interval_type = df_nona_interval[df_nona_interval["sourceCategory"] == type]
        
        sumVariable = len(df_nona_interval_type[df_nona_interval_type["has_variable"]==True])
        sumFilter = len(df_nona_interval_type[df_nona_interval_type["has_filter"]==True])
        sumOrderBy = len(df_nona_interval_type[df_nona_interval_type["has_orderby"]==True])
        sumLimit = len(df_nona_interval_type[df_nona_interval_type["has_limit"]==True])
        sumSelect = len(df_nona_interval_type[df_nona_interval_type["has_select"]==True])
        sumDistinct = len(df_nona_interval_type[df_nona_interval_type["has_distinct"]==True])
        sumToMultiSet = len(df_nona_interval_type[df_nona_interval_type["has_tomultiset"]==True])
        sumJoin = len(df_nona_interval_type[df_nona_interval_type["has_join"]==True])
        sumBGP = len(df_nona_interval_type[df_nona_interval_type["has_bgp"]==True])
        sumproject = len(df_nona_interval_type[df_nona_interval_type["has_project"]==True])
        sumUnion = len(df_nona_interval_type[df_nona_interval_type["has_union"]==True])
        sumValues = len(df_nona_interval_type[df_nona_interval_type["has_values"]==True])
        sumSlice = len(df_nona_interval_type[df_nona_interval_type["has_slice"]==True])
        sumSlice = len(df_nona_interval_type[df_nona_interval_type["has_offset"]==True])
        sumOffset = len(df_nona_interval_type[df_nona_interval_type["has_offset"]==True])

        data['I' + str(i) + " - " + type] = [sumVariable,sumValues,sumFilter,sumOrderBy,sumLimit,sumSelect,sumDistinct,sumSlice, sumToMultiSet,sumJoin,sumBGP,sumproject,sumUnion, sumOffset]

dpfr = pd.DataFrame(data=data, index=['Variable','Values','Filter','OrderBy','Limit','Select','Distinct','Slice', 'ToMultiSet','Join','BGP', 'project', 'Union',"Offset"])
dpfr

### Top 10 with triples

In [ ]:
df_nona_sort_by_triples = df_nona.sort_values(["triples"],  ascending=[False])
df_nona_sort_by_triples_out = df_nona_sort_by_triples.head(10)
df_nona_sort_by_triples_out

In [ ]:
%%script false --no-raise-error # disable cell

# Afficher la requête contenant le plus de triples
print(encoded2raw(
        df_nona_sort_by_triples_out.loc[53856]["anonymizedQuery"]
    ))

### Organic

Number of organic queries

In [ ]:
len(df_nona_organic)

Sum of all variables

In [ ]:
df_nona_organic["var_cpt"].sum()

Average number of variable per organic request

In [ ]:
df_nona_organic["var_cpt"].mean()

### Robotic

In [ ]:
len(df_nona_robotic)

Sum of all variables

In [ ]:
df_nona_robotic["var_cpt"].sum()

In [ ]:
df_nona_robotic["var_cpt"].mean()